In [ ]:
import glob, os, itertools


ocr_files = sorted(
    glob.glob("/content/ocr/**/*.json", recursive=True)
)
gt_files  = sorted(
    glob.glob("/content/gt/**/*.json",  recursive=True)
)

#if you’ve already built the lists and only need to purge the ellipsis:
ocr_files = [p for p in ocr_files if p is not Ellipsis]
gt_files  = [p for p in gt_files  if p is not Ellipsis]

# sanity‑check
print("OCR paths :", len(ocr_files))
print(" GT paths :", len(gt_files))
print("bad in ocr:", [p for p in ocr_files if not isinstance(p, (str, bytes, os.PathLike))])
print("bad in gt :", [p for p in gt_files  if not isinstance(p, (str, bytes, os.PathLike))])


OCR paths : 75
 GT paths : 75
bad in ocr: []
bad in gt : []


# Preprocessing and deterministic method along with merging block method

In [ ]:
# ------------------------------------------------------------
# 1.  Install deps (fast) –  comment out if already installed
# ------------------------------------------------------------
!pip install -q jiwer rapidfuzz

# ------------------------------------------------------------
# 2.  Imports & helpers
# ------------------------------------------------------------
import glob, json, unicodedata, re, os, math, time
from rapidfuzz.distance import Levenshtein  # 10‑20× faster than pure‑py
from jiwer import wer, cer

# ---------- tiny helpers ----------
def normalize(text: str) -> str:
    text = unicodedata.normalize("NFC", text)
    return re.sub(r"\s+", " ", text).strip()

def split_into_sents(text: str):
    """Very lightweight Gujarati / generic Indic sentence splitter."""
    if not text:
        return []
    # mark splits
    for delim in [".", "!", "?", "।"]:
        text = text.replace(delim, delim + "<SENT>")
    parts = text.split("<SENT>")
    return [p.strip() for p in parts if p.strip()]

# ---------- OCR block handling ----------
def ocr_block_text(block):
    """
    OCR JSON examples come as:
      {
        "block_text": "...",
        "lines": [{ "line_text": "..."}, ... ]
      }
    Strategy:
      * if 'lines' are dicts → join their 'line_text'
      * else fallback to 'block_text'
    """
    if "lines" in block and block["lines"]:
        first = block["lines"][0]
        if isinstance(first, dict) and "line_text" in first:
            return " ".join(normalize(l["line_text"]) for l in block["lines"])
        elif isinstance(first, str):
            return " ".join(normalize(l) for l in block["lines"])
    return normalize(block.get("block_text", ""))

# ---------- GT text extraction ----------
def extract_gt_lines(obj):
    """
    Recursively walk an arbitrary GT JSON object and
    collect every list-of-strings under a 'lines' key.
    Returns a list of *paragraph strings* (we'll sentence‑split later).
    """
    para = []
    if isinstance(obj, dict):
        # grab & join lines if they look like ['str', 'str', ...]
        if "lines" in obj and isinstance(obj["lines"], list) \
           and all(isinstance(x, str) for x in obj["lines"]):
            para.append(" ".join(normalize(x) for x in obj["lines"]))
        # recurse
        for v in obj.values():
            para.extend(extract_gt_lines(v))
    elif isinstance(obj, list):
        for item in obj:
            para.extend(extract_gt_lines(item))
    return para

# ---------- merge small OCR blocks ----------
def merge_small_blocks(blocks, thr=15):
    """
    Merge any block with <thr words into its neighbour (prev, or next if first).
    Keeps a 'merged_from' list of original indices (for provenance).
    """
    def wc(txt): return len(txt.split())
    merged = [{"txt": ocr_block_text(b), "merged_from": [i]}
              for i, b in enumerate(blocks)]
    i = 0
    while i < len(merged):
        if wc(merged[i]["txt"]) < thr:
            if len(merged) == 1:                        # only one block
                i += 1
            elif i == 0:                                # merge into next
                merged[i+1]["txt"] = merged[i]["txt"] + " " + merged[i+1]["txt"]
                merged[i+1]["merged_from"].extend(merged[i]["merged_from"])
                merged.pop(i)
            else:                                       # merge into previous
                merged[i-1]["txt"] += " " + merged[i]["txt"]
                merged[i-1]["merged_from"].extend(merged[i]["merged_from"])
                merged.pop(i)
        else:
            i += 1
    return merged

# ---------- fast Levenshtein wrapper ----------
def lev(a, b):
    return Levenshtein.distance(a, b)

# ---------- dynamic‑programming aligner ----------
def align_edit(ocr_segs, gt_segs):
    """
    Needleman‑Wunsch style DP on sentence lists.
    del / ins cost = len(segment)   (i.e. #characters)
    sub cost       = Levenshtein distance
    Returns list[(ocr_sent, gt_sent)] for aligned pairs (skips gaps).
    """
    M, N = len(ocr_segs), len(gt_segs)
    # DP tables
    dp  = [[0]*(N+1) for _ in range(M+1)]
    back= [[0]*(N+1) for _ in range(M+1)]   # 0=sub, 1=del, 2=ins

    for i in range(1, M+1):
        dp[i][0] = dp[i-1][0] + len(ocr_segs[i-1]); back[i][0]=1
    for j in range(1, N+1):
        dp[0][j] = dp[0][j-1] + len(gt_segs[j-1]);  back[0][j]=2

    cache = {}
    for i in range(1, M+1):
        for j in range(1, N+1):
            key = (i-1, j-1)
            if key not in cache:
                cache[key] = lev(ocr_segs[i-1], gt_segs[j-1])
            sub = dp[i-1][j-1] + cache[key]
            dele= dp[i-1][j]   + len(ocr_segs[i-1])
            ins = dp[i][j-1]   + len(gt_segs[j-1])
            dp[i][j], back[i][j] = min(
                (sub,0), (dele,1), (ins,2), key=lambda x:x[0])

    # traceback
    pairs=[]
    i,j=M,N
    while i>0 or j>0:
        b = back[i][j]
        if b==0:
            pairs.append((ocr_segs[i-1], gt_segs[j-1]))
            i-=1; j-=1
        elif b==1:
            i-=1                                   # skip OCR sentence
        else:
            j-=1                                   # skip GT sentence
    return pairs[::-1]

# ------------------------------------------------------------
# 3.  Collect filepaths
# ------------------------------------------------------------
OCR_DIR = "/content/ocr"
GT_DIR  = "/content/gt"

ocr_files = sorted(glob.glob(f"{OCR_DIR}/**/*.json", recursive=True))
gt_files  = sorted(glob.glob(f"{GT_DIR}/**/*.json",  recursive=True))
assert len(ocr_files)==len(gt_files), "OCR/GT file counts differ!"

print("OCR paths :", len(ocr_files))
print("GT  paths :", len(gt_files))

# ------------------------------------------------------------
# 4.  Build pages_data  (clean sentences only)
# ------------------------------------------------------------
pages_data=[]
t0=time.time()

for ocr_path, gt_path in zip(ocr_files, gt_files):
    # ---------- OCR ----------
    with open(ocr_path, encoding="utf‑8") as f:
        ocr_json = json.load(f)
    ocr_blocks_raw = ocr_json["ocr_output"]["blocks"]
    merged = merge_small_blocks(ocr_blocks_raw, thr=15)
    ocr_sent=[]
    for blk in merged:
        ocr_sent.extend(split_into_sents(normalize(blk["txt"])))

    # ---------- GT ----------
    with open(gt_path, encoding="utf‑8") as f:
        gt_json = json.load(f)
    gt_paras = extract_gt_lines(gt_json)
    gt_sent=[]
    for para in gt_paras:
        gt_sent.extend(split_into_sents(para))

    pages_data.append({"ocr_sent": ocr_sent, "gt_sent": gt_sent})

print(f"Pre‑processing finished\n  pages processed : {len(pages_data)}\n"
      f"  elapsed         : {time.time()-t0:,.1f}s")

# ------------------------------------------------------------
# 5.  Deterministic alignment
# ------------------------------------------------------------
aligned_pairs=[]
t0=time.time()
for page in pages_data:
    aligned_pairs.extend(align_edit(page["ocr_sent"], page["gt_sent"]))
print(f"\nDeterministic alignment completed\n"
      f"  aligned segment pairs : {len(aligned_pairs)}\n"
      f"  elapsed               : {time.time()-t0:,.1f}s\n")

# quick sanity peek
print("  sample pairs (first 5):")
for o,g in aligned_pairs[:5]:
    print("OCR:", o[:120])
    print(" GT:", g[:120])
    print("---")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 56.7 MB/s eta 0:00:00
OCR paths : 75
GT  paths : 75
Pre‑processing finished
  pages processed : 75
  elapsed         : 0.1s

Deterministic alignment completed
  aligned segment pairs : 1671
  elapsed               : 0.2s

  sample pairs (first 5):
OCR: ગાંધીજીનો અક્ષરદેહ એ સજ્જને પેલા લેખમાંથી જે ઉતારાનું ભાષાંતર કરીને મોકલ્યું છે તે આ રહ્યું :
 GT: એ સજ્જને પેલા લેખમાંથી જે ઉતારાનું ભાષાંતર કરીને મોકલ્યું છે તે આ રહ્યું :
---
OCR: ગાંધીના ધર્માંધ આધ્યાત્મિક સામ્રાજ્યવાદ અને પાશ્ચાત્ય સુધારાનો દ્વેષ એ બે વસ્તુમાં પાછા હઠતા હિંદુસ્તાનનું ચિહ્ન રહેલું 
 GT: ગાંધીના ધર્માંધ આધ્યાત્મિક સામ્રાજ્યવાદ અને પાશ્ચાત્ય સુધારાનો દ્વેષ એ બે વસ્તુમાં પાછા હઠતા હિંદુસ્તાનનું ચિહ્ન રહેલું 
---
OCR: એનો આદર્શ એટલે અસલની ગ્રામ- પંચાયતો , — જેની આર્થિક સ્વતંત્રતા એ બાહ્ય જગતથી અલિપ્ત રહેવામાં જ હતી , અને આને માટે ગાંધી 
 GT: એનો આદર્શ એટલે અસલની ગ્રામપંચાયતો, — જેની આર્થિક સ્વતંત્રતા એ બાહ્ય જગતથી અલિપ્ત રહેવામાં જ હતી, અને આને માટે ગાંધી કાંત
---
OC

In [ ]:
print(len(pages_data), "pages in pages_data")
for k in pages_data[0].keys():
    print("key:", k, "→", type(pages_data[0][k]),
          "len" if isinstance(pages_data[0][k], list) else "",
          len(pages_data[0][k]) if isinstance(pages_data[0][k], list) else "")


75 pages in pages_data
key: ocr_sent → <class 'list'> len 32
key: gt_sent → <class 'list'> len 31


In [ ]:
!pip -q install sentence_transformers faiss-cpu jiwer regex

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.7 MB/s eta 0:00:00


# Embedding based alignment using paraphrase-multilingual-MiniLM-L12-v2

In [ ]:
# ------------------------------------------------------------
# 2)  Embedding‑based (non‑deterministic) alignment
# ------------------------------------------------------------
#
# ‑ we embed **all** OCR/GT sentences at once
# ‑ we build two FAISS ‑IP (cosine‑sim) indices
# ‑ alignment is “reciprocal nearest neighbour” *within the same page*
# ------------------------------------------------------------


import faiss, time, numpy as np
from sentence_transformers import SentenceTransformer

# ------------------------------------------------------------
# 2‑a)  flatten the corpus  →  global sentence lists
# ------------------------------------------------------------
all_ocr_sent, all_gt_sent = [], []
idx_ocr, idx_gt          = [], []        #  ←  FIX: two empty lists

for pid, page in enumerate(pages_data):           # pages_data already built before
    for i, s in enumerate(page["ocr_sent"]):
        all_ocr_sent.append(s)
        idx_ocr.append((pid, i))

    for j, s in enumerate(page["gt_sent"]):
        all_gt_sent.append(s)
        idx_gt.append((pid, j))

print(f"  • OCR sentences : {len(all_ocr_sent):,}")
print(f"  • GT  sentences : {len(all_gt_sent):,}")

# ------------------------------------------------------------
# 2‑b)  Sentence embeddings  (multilingual SBERT)
# ------------------------------------------------------------
from sentence_transformers import SentenceTransformer


model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
def encode_batch(texts, bsz=64):
    vec = model.encode(texts,
                       batch_size=bsz,
                       convert_to_numpy=True,
                       show_progress_bar=True)
    # ℓ2 normalise  → cosine = dot
    return vec / np.linalg.norm(vec, axis=1, keepdims=True)

ocr_vec = encode_batch(all_ocr_sent)
gt_vec  = encode_batch(all_gt_sent)
dim     = ocr_vec.shape[1]

# ------------------------------------------------------------
# 2‑c)  FAISS indices  (inner‑product = cosine)
# ------------------------------------------------------------
index_gt  = faiss.IndexFlatIP(dim)
index_gt.add(gt_vec)                # search GT for each OCR

index_ocr = faiss.IndexFlatIP(dim)
index_ocr.add(ocr_vec)              # search OCR for each GT   (reciprocity test)

# pre‑compute GT → best OCR  (top‑1 is enough for reciprocity)
sim_gt2ocr, idx_gt2ocr = index_ocr.search(gt_vec, 1)
best_ocr_for_gt = idx_gt2ocr[:, 0]          # shape = (|GT|,)

# ------------------------------------------------------------
# 2‑d)  page‑local reciprocal‑NN alignment helper
# ------------------------------------------------------------
def page_alignment(pg_id, top_k=5, thresh=0.60):
    """
    Return list[(ocr_text, gt_text, cosine_sim)] for a single page
    using reciprocal‑NN + same‑page filter.
    """
    ocr_sent = pages_data[pg_id]["ocr_sent"]
    gt_sent  = pages_data[pg_id]["gt_sent"]
    if not ocr_sent or not gt_sent:
        return []

    # --- global indices that belong to this page -----------------
    gidx_ocr = [g for g,(p,_) in enumerate(idx_ocr) if p == pg_id]
    gidx_gt  = [g for g,(p,_) in enumerate(idx_gt)  if p == pg_id]
    map_g2local_gt = {g: j for j, g in enumerate(gidx_gt)}   # fast global→local

    # --- OCR → (top‑k) GT search --------------------------------
    sim_o2g, cand_o2g = index_gt.search(ocr_vec[gidx_ocr], top_k)  # (m, k)

    pairs = []
    for o_local, (g_candidates, s_candidates) in enumerate(zip(cand_o2g, sim_o2g)):
        for g_global, score in zip(g_candidates, s_candidates):
            if score < thresh:
                break                     # matrix is sorted – the rest are lower

            g_local = map_g2local_gt.get(int(g_global))  # None ⇒ sentence is on another page
            if g_local is None:
                continue

            # reciprocity
            if best_ocr_for_gt[g_global] == gidx_ocr[o_local]:
                pairs.append((
                    ocr_sent[o_local],
                    gt_sent[g_local],
                    float(score)
                ))
                break                     # keep only first mutual match

    return pairs

# ------------------------------------------------------------
# 2‑e)  run over all pages
# ------------------------------------------------------------
t0 = time.time()
aligned_pairs_emb = []
for pid in range(len(pages_data)):
    aligned_pairs_emb.extend(page_alignment(pid))

print(f"\nEmbedding alignment ✔ {len(aligned_pairs_emb)} pairs   "
      f"in {time.time()-t0:.1f}s")

# quick sanity peek
for o, g, s in aligned_pairs_emb[:3]:
    print(f"\nsim={s:.3f}\nOCR: {o[:120]}…\n GT: {g[:120]}…")


  • OCR sentences : 1,761
  • GT  sentences : 1,834


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/28 [00:00<?, ?it/s]

Batches:   0%|          | 0/29 [00:00<?, ?it/s]


Embedding alignment ✔ 733 pairs   in 0.2s

sim=0.899
OCR: ગાંધીજીનો અક્ષરદેહ એ સજ્જને પેલા લેખમાંથી જે ઉતારાનું ભાષાંતર કરીને મોકલ્યું છે તે આ રહ્યું :…
 GT: એ સજ્જને પેલા લેખમાંથી જે ઉતારાનું ભાષાંતર કરીને મોકલ્યું છે તે આ રહ્યું :…

sim=0.916
OCR: છતાં સાથે સાથે રોજી આપનાર રાજકાજ એ ચલાવી રહ્યો છે .…
 GT: છતાં સાથે સાથે રોજી આપનાર રાજકાજ એ ચલાવી રહ્યો છે.…

sim=0.867
OCR: આવો પ્રભાવશાળી પુરુષ જે રીતે પોતાની લડત ચલાવી રહેલ છે તેનું અવલોકન કરનારને પાશ્ચાત્ય સુધારા વિશેના તેના વિચારો ભ્રમિત લા…
 GT: આ બધું મેળવવા માગવું એ પોતાના મૂળ સિદ્ધાંતોને ઉલટાવવા સરખું છે … આવો પ્રભાવશાળી પુરુષ જે રીતે પોતાની લડત ચલાવી રહેલ છે ત…


# Embedding based alignment using l3cube-pune/indic-sentence-similarity-sbert

In [ ]:
# install deps (only once)
!pip install -qU sentence_transformers faiss-cpu

from sentence_transformers import SentenceTransformer
import faiss, numpy as np, time

# 1) Load a true SBERT‐style model fine‑tuned for Indic STS
MODEL_NAME = "l3cube-pune/indic-sentence-similarity-sbert"
model      = SentenceTransformer(MODEL_NAME)

# 2) Precompute and store embeddings (with L2‑norm) for every page
for p in pages_data:
    o_sent = p.get("ocr_sent", [])
    g_sent = p.get("gt_sent", [])
    # encode + normalize
    if o_sent:
        o_emb = model.encode(o_sent, batch_size=64, convert_to_numpy=True, show_progress_bar=False)
        faiss.normalize_L2(o_emb)
    else:
        o_emb = np.zeros((0, model.get_sentence_embedding_dimension()), dtype=np.float32)
    if g_sent:
        g_emb = model.encode(g_sent, batch_size=64, convert_to_numpy=True, show_progress_bar=False)
        faiss.normalize_L2(g_emb)
    else:
        g_emb = np.zeros((0, model.get_sentence_embedding_dimension()), dtype=np.float32)

    # stash back on the page dict
    p["o_emb"], p["g_emb"] = o_emb, g_emb

def align_page(p, k=5, th=0.6):
    """
    Reciprocal‐NN over precomputed embeddings p['o_emb'] vs p['g_emb'].
    Returns list of (ocr_sentence, gt_sentence, cosine_sim).
    """
    o_emb, g_emb = p["o_emb"], p["g_emb"]
    o_sent, g_sent = p.get("ocr_sent", []), p.get("gt_sent", [])
    if o_emb.shape[0]==0 or g_emb.shape[0]==0:
        return []

    # build the index on GT
    idx = faiss.IndexFlatIP(o_emb.shape[1])
    idx.add(g_emb)

    # for each OCR embedding, fetch top‐k GT
    D, I = idx.search(o_emb, k)   # D=cosines, I=indices

    # find, for each GT, the single best OCR partner
    best_ocr_for_gt = np.full(g_emb.shape[0], -1, dtype=int)
    best_sim        = np.full(g_emb.shape[0], -1.0, dtype=float)
    for o_idx, (sims, g_idxs) in enumerate(zip(D, I)):
        for sim, g_idx in zip(sims, g_idxs):
            if sim < th:
                break
            if sim > best_sim[g_idx]:
                best_sim[g_idx]        = sim
                best_ocr_for_gt[g_idx] = o_idx

    # now pick reciprocal + monotone matches
    pairs, last_g = [], -1
    for o_idx, (sims, g_idxs) in enumerate(zip(D, I)):
        for sim, g_idx in zip(sims, g_idxs):
            if sim < th:
                break
            if best_ocr_for_gt[g_idx] == o_idx and g_idx > last_g:
                pairs.append((o_sent[o_idx], g_sent[g_idx], float(sim)))
                last_g = g_idx
                break

    return pairs

# 3) Run it over all pages
t0 = time.time()
emb_pairs = []
for p in pages_data:
    emb_pairs.extend(align_page(p, k=5, th=0.6))
print(f"✔ Aligned {len(emb_pairs)} pairs in {time.time()-t0:.1f}s")

# Quick sanity‐check
for o, g, sim in emb_pairs[:3]:
    print(f"sim={sim:.3f}\n OCR: {o}\n GT : {g}\n---")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✔ Aligned 1431 pairs in 0.0s
sim=0.736
 OCR: ગાંધીજીનો અક્ષરદેહ એ સજ્જને પેલા લેખમાંથી જે ઉતારાનું ભાષાંતર કરીને મોકલ્યું છે તે આ રહ્યું :
 GT : એ સજ્જને પેલા લેખમાંથી જે ઉતારાનું ભાષાંતર કરીને મોકલ્યું છે તે આ રહ્યું :
---
sim=1.000
 OCR: ગાંધીના ધર્માંધ આધ્યાત્મિક સામ્રાજ્યવાદ અને પાશ્ચાત્ય સુધારાનો દ્વેષ એ બે વસ્તુમાં પાછા હઠતા હિંદુસ્તાનનું ચિહ્ન રહેલું છે .
 GT : ગાંધીના ધર્માંધ આધ્યાત્મિક સામ્રાજ્યવાદ અને પાશ્ચાત્ય સુધારાનો દ્વેષ એ બે વસ્તુમાં પાછા હઠતા હિંદુસ્તાનનું ચિહ્ન રહેલું છે.
---
sim=0.985
 OCR: એનો આદર્શ એટલે અસલની ગ્રામ- પંચાયતો , — જેની આર્થિક સ્વતંત્રતા એ બાહ્ય જગતથી અલિપ્ત રહેવામાં જ હતી , અને આને માટે ગાંધી કાંતવાની હિમાયત કરે છે .
 GT : એનો આદર્શ એટલે અસલની ગ્રામપંચાયતો, — જેની આર્થિક સ્વતંત્રતા એ બાહ્ય જગતથી અલિપ્ત રહેવામાં જ હતી, અને આને માટે ગાંધી કાંતવાની હિમાયત કરે છે.
---


# Evaluation of all methods

In [ ]:
# ────────────────────────────────────────────────────────────────
# 0) Rename your existing alignment outputs to the expected names
# ────────────────────────────────────────────────────────────────
#   aligned_pairs       ← [(ocr_text, gt_text), …]               # from your edit‑distance pipeline
#   aligned_pairs_emb   ← [(ocr_text, gt_text, cos_sim), …]      # from your MiniLM pipeline
#   emb_pairs           ← [(ocr_text, gt_text, cos_sim), …]      # from your IndicSBERT pipeline

aligned_pairs_edit      = aligned_pairs       # deterministic (edit‑distance) results
emb_pairs_miniLM        = aligned_pairs_emb   # MiniLM multilingual results
emb_pairs_indicSBERT    = emb_pairs           # IndicSBERT‑v2‑SS results

# ────────────────────────────────────────────────────────────────
# 1) Imports & helper functions
# ────────────────────────────────────────────────────────────────
import time
from statistics import mean, pstdev
from jiwer import wer, cer

def lev_sim(a: str, b: str) -> float:
    """
    Compute normalized Levenshtein‑distance similarity in [0,1].
    1.0 if both strings empty; otherwise 1 – (edit_distance / max_len).
    """
    if not (a or b):
        return 1.0
    # initialize previous row [0,1,2,…,len(b)]
    prev = list(range(len(b) + 1))
    for i, ca in enumerate(a, 1):
        curr = [i]
        for j, cb in enumerate(b, 1):
            cost_sub = prev[j-1] + (ca != cb)
            cost_del = prev[j]   + 1
            cost_ins = curr[j-1] + 1
            curr.append(min(cost_sub, cost_del, cost_ins))
        prev = curr
    dist = prev[-1]
    return 1 - dist / max(len(a), len(b))

def accumulate(det_pairs, emb_pairs):
    """
    Given two lists of aligned pairs:
      det_pairs: list of (ocr, gt)
      emb_pairs: list of (ocr, gt, sim_score)
    Returns a dict with keys 'sim', 'wer', 'cer', each mapping to a list of values.
    """
    stats = {"sim": [], "wer": [], "cer": []}

    # 1. process deterministic (edit‑distance) alignments
    for ocr_text, gt_text in det_pairs:
        # similarity via normalized Levenshtein
        s = lev_sim(ocr_text, gt_text)
        stats["sim"].append(s)
        # WER & CER via jiwer, only if GT text is non‑empty
        if gt_text.strip():
            stats["wer"].append(wer(gt_text, ocr_text))
            stats["cer"].append(cer(gt_text, ocr_text))

    # 2. process embedding‑based alignments
    for ocr_text, gt_text, sim_score in emb_pairs:
        stats["sim"].append(sim_score)
        if gt_text.strip():
            stats["wer"].append(wer(gt_text, ocr_text))
            stats["cer"].append(cer(gt_text, ocr_text))

    return stats

def summarise(name, stats):
    """
    Print a summary for one alignment method:
      - number of aligned pairs
      - mean ± std of similarity, WER, CER
      - % of alignments above similarity thresholds
    """
    sims = stats["sim"]
    wers = stats["wer"]
    cers = stats["cer"]

    print(f"\n── {name} ──  pairs: {len(sims)}")
    print(f" similarity : {mean(sims):.3f} ± {pstdev(sims):.3f}")
    print(f" WER        : {mean(wers):.3f} ± {pstdev(wers):.3f}")
    print(f" CER        : {mean(cers):.3f} ± {pstdev(cers):.3f}")
    for thr in (0.50, 0.75, 0.95):
        pct = sum(sim >= thr for sim in sims) * 100 / len(sims)
        print(f" ≥{thr:.2f} sim : {pct:5.1f}%")

# ────────────────────────────────────────────────────────────────
# 2) Run the evaluation
# ────────────────────────────────────────────────────────────────
t0 = time.time()

# accumulate stats for each method
stats_edit       = accumulate(aligned_pairs_edit,      [])
stats_miniLM     = accumulate([], emb_pairs_miniLM)
stats_indicSBERT = accumulate([], emb_pairs_indicSBERT)

# report runtime
print(f"\n⏱ Evaluation completed in {time.time() - t0:.2f}s")

# print detailed summaries
summarise("Deterministic (edit‑distance)",    stats_edit)
summarise("MiniLM multilingual (v1)",         stats_miniLM)
summarise("IndicSBERT‑v2‑SS (transformer)",   stats_indicSBERT)



⏱ Evaluation completed in 4.96s

── Deterministic (edit‑distance) ──  pairs: 1671
 similarity : 0.869 ± 0.174
 WER        : 0.573 ± 0.602
 CER        : 0.230 ± 1.422
 ≥0.50 sim :  95.4%
 ≥0.75 sim :  86.1%
 ≥0.95 sim :  37.8%

── MiniLM multilingual (v1) ──  pairs: 733
 similarity : 0.931 ± 0.070
 WER        : 0.589 ± 0.617
 CER        : 0.155 ± 0.337
 ≥0.50 sim : 100.0%
 ≥0.75 sim :  98.4%
 ≥0.95 sim :  52.0%

── IndicSBERT‑v2‑SS (transformer) ──  pairs: 1346
 similarity : 0.935 ± 0.082
 WER        : 0.516 ± 0.526
 CER        : 0.129 ± 0.246
 ≥0.50 sim : 100.0%
 ≥0.75 sim :  96.2%
 ≥0.95 sim :  58.6%
